# Conversational Threads

Many LLM applications have a chatbot-like interface in which the user and the LLM application engage in a multi-turn conversation. In order to track these conversations, you can use the Threads feature in LangSmith.

This is relevant to our RAG application, which should maintain context from prior conversations with users.

### Setup

In [ ]:
# You can set them inline
import os

from test import anthropic_client

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"  # If you don't set this, traces will go to the Default project

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv()

True

### Group traces into threads


A Thread is a sequence of traces representing a single conversation. Each response is represented as its own trace, but these traces are linked together by being part of the same thread.

To associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.

The key value is the unique identifier for that conversation. The key name should be one of:

- session_id
- thread_id
- conversation_id.

The value should be a UUID.

In [2]:
import uuid
thread_id = uuid.uuid4()

In [3]:
from langsmith import traceable
from openai import OpenAI
from anthropic import Anthropic
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

# openai_client = OpenAI()
anthropic_client = Anthropic()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    """
    messages = [
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_claude(messages, rag_system_prompt)

@traceable(run_type="llm")
def call_claude(
    messages: List[dict], system: str, model: str = "claude-3-5-haiku-latest", temperature: float = 0.0
) -> str:
    return anthropic_client.messages.create(
        model=model,
        messages=messages,
        system=system,
        temperature=temperature,
        max_tokens=1024
    )

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.content[0].text


USER_AGENT environment variable not set, consider setting it to identify your requests.
/Users/manan/PycharmProjects/langsmith-course/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Now let's run our application twice with this thread_id

In [4]:
question = "How do I add metadata to a Trace?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

I apologize, but I cannot find information in the provided context that directly answers how to add metadata to a Trace. The context appears to contain academic publications, research project details, and conference presentations, but does not discuss trace metadata or tracing systems. To get an accurate answer about adding metadata to a Trace, I would recommend consulting technical documentation specific to the tracing system you are using.


In [5]:
question = "How can I add tags to a Trace?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"thread_id": thread_id}})
print(ai_answer)

I apologize, but I cannot find information about adding tags to a Trace in the provided context. The context appears to contain academic publications and research papers across various fields like physics, environmental science, and biochemistry, but none discuss trace tagging. To accurately answer your question, I would need additional context or documentation specific to tracing systems or software.


### Let's take a look in LangSmith!